<a href="https://colab.research.google.com/github/alfonsorinconbarrera/alfonsorinconbarrera/blob/main/pdf_a_docx_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Este código convierte documentos Pdf a Docx, eliminando saltos de secciones.
# Puede tardar algo más de 3 minutos dependiendo del tamaño del archivo

# Realiza una limpieza profunda de elementos de sección donde suelen estar los
# encabezados y pies de página.  Este comportamiento ocurre porque el código:
# -Elimina elementos vacíos que a menudo contienen marcadores de formato
# -Simplifica la estructura del documento al establecer márgenes consistentes

# Instalar las bibliotecas necesarias
!pip install pdf2docx python-docx Pillow pymupdf

import fitz
from pdf2docx import Converter
from docx import Document
from google.colab import files
import os
import tempfile
import shutil

# Subir el archivo PDF
print("Por favor, sube tu archivo PDF cuando aparezca el botón...")
uploaded = files.upload()

# Obtener el nombre del archivo subido
pdf_filename = list(uploaded.keys())[0]

def preprocesar_pdf(ruta_pdf_original):
    """
    Crea una copia del PDF con imágenes pre-procesadas
    """
    temp_pdf = os.path.splitext(ruta_pdf_original)[0] + '_temp.pdf'

    # Abrir el PDF original
    doc = fitz.open(ruta_pdf_original)

    # Crear un nuevo documento
    new_doc = fitz.open()

    # Copiar cada página
    for page in doc:
        new_page = new_doc.new_page(width=page.rect.width, height=page.rect.height)
        # Copiar solo el contenido de texto
        new_page.show_pdf_page(new_page.rect, doc, page.number)

    # Guardar el nuevo PDF
    new_doc.save(temp_pdf)
    new_doc.close()
    doc.close()

    return temp_pdf

def limpiar_documento_word(ruta_docx):
    """
    Elimina saltos de sección y columna de un documento Word
    """
    doc = Document(ruta_docx)

    # Procesar cada párrafo
    for paragraph in doc.paragraphs:
        if paragraph.text.strip() == "":
            p = paragraph._element
            p.getparent().remove(p)

    # Procesar cada sección
    for section in doc.sections:
        section.start_type = 0
        section.left_margin = 457200
        section.right_margin = 457200
        section.top_margin = 457200
        section.bottom_margin = 457200

    doc.save(ruta_docx)

def convertir_pdf_a_word(ruta_pdf):
    """
    Convierte PDF a Word usando pre-procesamiento
    """
    try:
        # Pre-procesar el PDF
        print("Pre-procesando el PDF...")
        temp_pdf = preprocesar_pdf(ruta_pdf)

        # Crear nombre para el archivo Word
        ruta_word = os.path.splitext(ruta_pdf)[0] + '.docx'

        print("Convirtiendo a Word...")
        # Convertir el PDF pre-procesado a Word
        cv = Converter(temp_pdf)
        cv.convert(ruta_word, start=0, start_from=0)
        cv.close()

        print("Limpiando formato...")
        # Limpiar el documento
        limpiar_documento_word(ruta_word)

        # Eliminar archivo temporal
        if os.path.exists(temp_pdf):
            os.remove(temp_pdf)

        print(f"Conversión completada. Descargando: {ruta_word}")
        files.download(ruta_word)

    except Exception as e:
        print(f"Error durante el proceso: {str(e)}")
        try:
            # Eliminar archivos temporales si existen
            if os.path.exists(temp_pdf):
                os.remove(temp_pdf)
        except:
            pass

# Ejecutar la conversión
if pdf_filename.lower().endswith('.pdf'):
    print(f"Procesando {pdf_filename}...")
    convertir_pdf_a_word(pdf_filename)
else:
    print("Por favor, sube un archivo PDF válido.")

Por favor, sube tu archivo PDF cuando aparezca el botón...


Saving PRESENTACION_CC-INDUSTRIA.pdf to PRESENTACION_CC-INDUSTRIA.pdf
Procesando PRESENTACION_CC-INDUSTRIA.pdf...
Pre-procesando el PDF...
Convirtiendo a Word...


Limpiando formato...
Conversión completada. Descargando: PRESENTACION_CC-INDUSTRIA.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>